This code is a bit of a mess, but in essence is quite simple. When trying to predict optimally for profit we often know exactly the benifits of choosing a class over another and the class_weight is usually used in these cases. But I found it to be sub optimal although very close. So instead of using entropy or gini this decision tree takes an input of savings and collateral (profit/loss) and chooses the nodes to maximise profits. It's very flexible and can be adjusted to work with whatever your model needs.

The output for predicting is a dictionary, and so the random forest will simply create a list of dictionaries and vote on the final prediction.

Major con is that this is wildly inefficient, a single decision tree is taking me about 3 minutes to train on a 30 by 200,000 dataset.

In [1]:
import pandas as pd
import numpy as np
import math

In [13]:
class dt():
    def __init__(self, **kwargs):
        self.split_d = {}
        self.saving = 8
        self.collateral = 72
        self.max_depth = kwargs.get('max_depth', 2)
        self.random_subset = kwargs.get('random_subset', 1)
        if (self.random_subset > 1) or (self.random_subset <= 0):
            print('random_subset must be a number between 0 and 1.')
    
    # profit function
    def prof(self, array):
        if array.shape[0] == 0:
            return 0
        return ((array[array == 1].shape[0] * self.collateral) - (array.shape[0] * self.saving))
    
    # profit gain function
    def prof_gain(self, X, y, split_point):
        return self.prop2(X, split_point)*self.prof(y[X <= split_point])

    # find best split
    def fbs(self, X, y):
        # top split is [profit upper, profit lower, feature (int), feature split]
        top_split = np.zeros(4)
        # cycle through each feature
        random_features = np.random.choice(X.shape[1], size=math.ceil(X.shape[1]*self.random_subset), replace=False)
        print(random_features)
        #random_rows = an_array[random_indices, :]
        for i, x in zip(random_features, X.T[random_features,:]):
            # cycle through each feature split
            for f_s in np.unique(x):
                result = self.prof(y[X.T[i] >= f_s])
                if result > top_split[0]:
                    top_split = [result, i, f_s, 1]
                result = self.prof(y[X.T[i] < f_s])
                if result > top_split[0]:
                    top_split = [result, i, f_s, -1]
        # returns (profit, feature number, feature split)
        #print('top split', top_split)
        return top_split

    def fit(self, X, y):
        layer_list = [[]]
        
        self.split_d = {}

        finished = False

        split_id = 0

        # each list in layer_list holds the splits for that layer
        # a split has (X for that split, y for that split, feature number, feature split, split_id, direction of split)
        layer_list[0].append((X, y, 'null', 'null', split_id, 'null'))

        layer = 0

        # make the decision tree
        while not finished:
            # add a new layer
            layer_list.append([])
            layer += 1
            for split in layer_list[layer-1]:
                
                direction = split[5]
                
                split_id_temp = split[4]
                
                feature_number = split[2]
                feature_split = split[3]

                X_temp = split[0]
                y_temp = split[1]
    
                # if only one class left then move on
                if (len(np.unique(y_temp)) < 2) or (layer > self.max_depth):
                    # make a leaf node with format [0, 0, 0, class_id, finished]
                    if direction == 0:
                        self.split_d[split_id_temp] = [0, 0, 0, 0, 0, 0, 1]
                    else:
                        self.split_d[split_id_temp] = [0, 1, 0, 0, 0, 0, 1]
                    continue

                # add lower split in format [profit for upper split,, profit for lower pslit, feature (int), feature split, lower split to id, upper split to id, finished]
                split_id += 1
                split_temp = self.fbs(X_temp, y_temp)
                profit = split_temp[0]
                feature = split_temp[1]
                feature_split = split_temp[2]
                direction2 = split_temp[3]
                if direction2 == 1:
                    layer_list[layer].append((X_temp[X_temp[:,feature] < feature_split], y_temp[X_temp[:,feature] < feature_split], feature, feature_split, split_id, 0))
                    split_id += 1
                    # add upper split (the (direction2 +1)/2 swaps the split id around depending on the direction)
                    layer_list[layer].append((X_temp[X_temp[:, feature] >= feature_split], y_temp[X_temp[:,feature] >= feature_split], feature, feature_split, split_id, 1))
                    self.split_d[split_id_temp] = (profit, feature, feature_split, split_id-1, split_id, direction2, 0)
                else:
                    layer_list[layer].append((X_temp[X_temp[:,feature] >= feature_split], y_temp[X_temp[:,feature] >= feature_split], feature, feature_split, split_id, 0))
                    split_id += 1
                    # add upper split (the (direction2 +1)/2 swaps the split id around depending on the direction)
                    layer_list[layer].append((X_temp[X_temp[:, feature] < feature_split], y_temp[X_temp[:,feature] < feature_split], feature, feature_split, split_id, 1))
                    self.split_d[split_id_temp] = (profit, feature, feature_split, split_id-1, split_id, direction2, 0)


            if len(layer_list[-1]) == 0:
                finished = True
                
    def predict(self, array):
        result_array = []

        for x in array:
            #print('\n', x)
            finished = False
            split_id = 0
            while finished == False:
                result = self.split_d[split_id]
                
                leaf_node = result[6]
                
                feature_number = result[1]
                
                feature_split = result[2]
                
                direction = result[5]
                
                top_split_id = result[3]
                
                bottom_split_id = result[4]

                # if leaf node then end and print class
                if leaf_node == 1:
                    result_array.append(feature_number)
                    #print('Complete, cat is', feature_number)
                    break

                # move on to the next split
                if direction == 1:
                    #print('checking if',(x[feature_number]), 'greater or equal to', (feature_split))
                    if x[feature_number] >= feature_split:
                        split_id = bottom_split_id
                        #print('true, moving to', bottom_split_id)
                    else:
                        split_id = top_split_id
                        #print('false, moving to', top_split_id)
                else:
                    #print('checking if',(x[feature_number]), 'strictly less than', (feature_split))
                    if x[feature_number] < feature_split:
                        split_id = bottom_split_id
                        #print('true, moving to', bottom_split_id)
                    else:
                        split_id = top_split_id
                        #print('false, moving to', top_split_id)

        return np.array(result_array)

In [14]:
class rf():
    def __init__(self, **kwargs):
        self.split_d_list = []
        self.saving = 8
        self.collateral = 72
        self.max_depth = kwargs.get('max_depth', 2)
        self.random_subset = kwargs.get('random_subset', 1)
        if (self.random_subset > 1) or (self.random_subset <= 0):
            print('random_subset must be a number between 0 and 1.')
        self.n = kwargs.get('n', 10)
        
    def fit(self, X, y):
        tree = dt(max_depth=self.max_depth, random_subset=self.random_subset)
        for i in range(self.n):
            #print('tree', i+1)
            tree.fit(X ,y)
            self.split_d_list.append(tree.split_d)
            
    def tree_predict(self, array, number):
        result_array = []

        for x in array:
            #print('\n', x)
            finished = False
            split_id = 0
            split_d = self.split_d_list[number]
            while finished == False:
                result = split_d[split_id]
                
                leaf_node = result[6]
                
                feature_number = result[1]
                
                feature_split = result[2]
                
                direction = result[5]
                
                top_split_id = result[3]
                
                bottom_split_id = result[4]

                # if leaf node then end and print class
                if leaf_node == 1:
                    result_array.append(feature_number)
                    #print('Complete, cat is', feature_number)
                    break

                # move on to the next split
                if direction == 1:
                    #print('checking if',(x[feature_number]), 'greater or equal to', (feature_split))
                    if x[feature_number] >= feature_split:
                        split_id = bottom_split_id
                        #print('true, moving to', bottom_split_id)
                    else:
                        split_id = top_split_id
                        #print('false, moving to', top_split_id)
                else:
                    #print('checking if',(x[feature_number]), 'strictly less than', (feature_split))
                    if x[feature_number] < feature_split:
                        split_id = bottom_split_id
                        #print('true, moving to', bottom_split_id)
                    else:
                        split_id = top_split_id
                        #print('false, moving to', top_split_id)
                
        return np.array(result_array)
        
    def predict(self, array):
        result = np.zeros(shape=(self.n, np.array(array).shape[0]))
        for number in range(self.n):
            result[number, :] = self.tree_predict(array, number)
        return np.round(result.mean(axis=0))

In [15]:
from sklearn.datasets import make_blobs
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt
X, y = make_blobs(n_samples=20, random_state=42, n_features=4, cluster_std=3, centers=2)

In [16]:
df = pd.DataFrame(X, columns=['x1', 'x2', 'x3', 'x4'])

df['y'] = y

#col = df['y'].map({1:'r', 2:'b', 0:'g'})

#df.plot(kind='scatter', x='x1', y='x2', c=col)

In [17]:
model = dt(max_depth=5, random_subset=0.4)

model.fit(X, y)

model.predict([[-7, -10, 1, 1], [5, 0, 10, 10], [-5, 10, -10, -10]])

[0 1]


array([1, 1, 0])

In [18]:
model = rf(max_depth=4, random_subset=0.5, n=5)

model.fit(X, y)

model.predict([[-7, -10, 1, 1], [5, 0, 10, 10], [-5, 10, -10, -10]])

[1 2]
[1 3]
[3 1]
[3 2]
[1 0]


array([1., 1., 0.])